In [1]:
import numpy as np 
from MPHP import MPHP
from MHP import MHP

import matplotlib.pyplot as plt
%matplotlib inline 

In [17]:
m = np.array([0.2, 0.0, 0.0])
a = np.array([[0.1, 0.0, 0.0], 
              [0.9, 0.0, 0.0],
              [0.0, 0.9, 0.0]])
w = 3.1

P = MHP(mu=m, alpha=a, omega=w)
data=P.generate_seq(1000)
len(data)

Max eigenvalue: 0.10000


531

In [5]:
m = np.array([0.2, 0.0, 0.0])
a = np.array([[0.1, 0.0, 0.0], 
              [0.9, 0.0, 0.0],
              [0.0, 0.9, 0.0]])
w = 3.1

PP = MPHP(mu=m, alpha=a, omega=w)
datap = PP.generate_seq(1000)
len(datap)

Max eigenvalue: 0.10000


699

In [ ]:
PP.mu_day

In [ ]:
P.alpha

In [ ]:
datap = PP.generate_seq(60)
len(datap)

In [ ]:
len(P.data)

In [ ]:
delta = np.diff(P.data[:, 0])

In [ ]:
np.mean(delta)

In [ ]:
window = 1
parent = 0
child = 1
(delta <= window) & (P.data[:, 1]==parent) 

In [ ]:
data = P.data[25:35]
window=1
count = 0
parent=0
child=1
for i in range(len(data)):
    count += count_from_time(data, i, window, parent, child)
count

In [ ]:
def excitation_rate(data, window, parent, child):
    '''Number of child events per parent event
    
    window should be sig less than avg rate of child event
    should dep on omega -- see how alpha behaves, total contribution
    '''
    def count_from_time(data, i, window, parent, child, count=0, prev_diff=0, step=1):
    
        if (i+step) < len(data):
            diff = prev_diff + data[i+step, 0] - data[i, 0]
        else:
            return count

        if diff < window:
            if data[i, 1]==parent and data[i+step, 1]==child:
                count += 1
            count = count_from_time(data, i, window, parent, child, count=count, prev_diff=diff, step=step+1)

        return count

    count = 0
    for i in range(len(data)):
        count += count_from_time(data, i, window, parent, child)
        
    return float(count)/np.sum(data[:, 1]==parent)
        

In [ ]:
excitation_rate(P.data, window=0.5, parent=0, child=1)

In [ ]:
prop_child_events(P.data, window=0.5, parent=1, child=2)

In [ ]:
# def EM(Ahat, mhat, mhatday, omega, seq=[], a=[1, 1, 1, 1, 1, 5, 5], smx=None, tmx=None, regularize=False,
#        Tm=-1, maxiter=100, epsilon=0.01, verbose=True):
#     '''implements MAP EM. Optional to regularize with `smx` and `tmx` matrix (shape=(dim,dim)).
#     In general, the `tmx` matrix is a pseudocount of parent events from column j,
#     and the `smx` matrix is a pseudocount of child events from column j -> i, 
#     however, for more details/usage see https://stmorse.github.io/docs/orc-thesis.pdf'''

#     # if no sequence passed, uses class instance data
#     if len(seq) == 0:
#         seq = data

#     N = len(seq)
#     day = (np.floor(seq[:, 0]) % 7).astype(int)
#     dim = mhat.shape[0]
#     Tm = float(seq[-1, 0]) if Tm < 0 else float(Tm)
#     sequ = seq[:, 1].astype(int)

#     p_ii = np.random.uniform(0.01, 0.99, size=N)
#     p_ij = np.random.uniform(0.01, 0.99, size=(N, N))

#     # PRECOMPUTATIONS

#     # diffs[i,j] = t_i - t_j for j < i (o.w. zero)
#     diffs = pairwise_distances(np.array([seq[:, 0]]).T, metric='euclidean')
#     diffs[np.triu_indices(N)] = 0

#     # kern[i,j] = omega*np.exp(-omega*diffs[i,j])
#     kern = omega * np.exp(-omega * diffs)

#     colidx = np.tile(sequ.reshape((1, N)), (N, 1))
#     rowidx = np.tile(sequ.reshape((N, 1)), (1, N))

#     # approx of Gt sum in a_{uu'} denom **
#     seqcnts = np.array([len(np.where(sequ == i)[0]) for i in range(dim)])
#     seqcnts = np.tile(seqcnts, (dim, 1))

#     # returns sum of all pmat vals where u_i=a, u_j=b
#     # *IF* pmat upper tri set to zero, this is
#     # \sum_{u_i=u}\sum_{u_j=u', j<i} p_{ij}
#     def sum_pij(a, b):
#         c = cartesian([np.where(seq[:, 1] == int(a))[0], np.where(seq[:, 1] == int(b))[0]])
#         return np.sum(p_ij[c[:, 0], c[:, 1]])
#     vp = np.vectorize(sum_pij)

#     # \int_0^t g(t') dt' with g(t)=we^{-wt}
#     # def G(t): return 1 - np.exp(-omega * t)
#     #   vg = np.vectorize(G)
#     # Gdenom = np.array([np.sum(vg(diffs[-1,np.where(seq[:,1]==i)])) for i in range(dim)])

#     k = 0
#     old_LL = -10000

#     while k < maxiter:
#         Auu = Ahat[rowidx, colidx] #ahat[i, j] = a_ui, uj
#         ag = np.multiply(Auu, kern)
#         ag[np.triu_indices(N)] = 0

#         # compute m_{u_i}
#         mu = mhat[sequ]

#         # compute delta_{d_i}
#         mu_day = mhatday[day]

#         # compute total rates of u_i at time i
#         rates = mu*mu_day + np.sum(ag, axis=1)

#         # compute matrix of p_ii and p_ij  (keep separate for later computations)
#         p_ij = np.divide(ag, np.tile(np.array([rates]).T, (1, N)))
#         p_ii = np.divide(mu, rates)

#         # compute mhat:  mhat_u = (\sum_{u_i=u} p_ii) / T
#         mhat = np.array([np.sum(p_ii[np.where(seq[:, 1] == i)])
#                          for i in range(dim)]) / Tm

#         mhatday = np.array([np.divide(np.sum(p_ii[np.where(day == i)]) + a[i] - 1, np.sum(p_ii)/7 + a[i] - 1) for i in range(7)])

#         # ahat_{u,u'} = (\sum_{u_i=u}\sum_{u_j=u', j<i} p_ij) / \sum_{u_j=u'} G(T-t_j)
#         # approximate with G(T-T_j) = 1
#         if regularize:
#             Ahat = np.divide(np.fromfunction(lambda i, j: vp(i, j), (dim, dim)) + (smx - 1),
#                              seqcnts + tmx)
#         else:
#             Ahat = np.divide(np.fromfunction(lambda i, j: vp(i, j), (dim, dim)),
#                              seqcnts)

#         if k % 10 == 0:
#             try:
#                 term1 = np.sum(np.log(rates))
#             except:
#                 print('Log error!')
#             term2 = Tm * np.sum(mhat)
#             term3 = np.sum(np.sum(Ahat[u, int(seq[j, 1])] for j in range(N)) for u in range(dim))
#             #new_LL = (1./N) * (term1 - term2 - term3)
#             new_LL = (1. / N) * (term1 - term3)
#             if abs(new_LL - old_LL) <= epsilon:
#                 if verbose:
#                     print('Reached stopping criterion. (Old: %1.3f New: %1.3f)' % (old_LL, new_LL))
#                     alpha = Ahat
#                     mu = mhat
#                     mu_day = mhatday
#                 return Ahat, mhat, mhatday
#             if verbose:
#                 print('After ITER %d (old: %1.3f new: %1.3f)' % (k, old_LL, new_LL))
#                 print(' terms %1.4f, %1.4f, %1.4f' % (term1, term2, term3))

#             old_LL = new_LL

#         k += 1

#     if verbose:
#         print('Reached max iter (%d).' % maxiter)

#     alpha = Ahat
#     mu = mhat
#     mu_day = mhatday
#     return Ahat, mhat, mhatday
